In [2]:
from typing import Tuple, Callable


## Summary of sessions 1 and 2

In the first session we came up with a simple problem that nevertheless captures many of ideas involved in
modern machine learning: finding the optimal coefficients of a linear model for predicting the math grade
of a student from the time they spent studying in the lyzeum2.

We saw that generally for such problems, we will need to find the minima (or maxima) of rather
complicated functions. This brought us to the topic of _optimization_. Sometimes, for simple problems,
optimization can be performed analytically - this means finding an exact solution by solving a set of equations.
However, for the majority of problems the optimality equations are way too complicated to be addressed analytically,
and we need to resort to _numerical optimization_. This means finding the optimal values through a program - a
sequence of calculations.

We sketched our first program for solving a very simplified optimization - finding the minimum of a one dimensional
function. The sketch evolved around the following idea:

Given a function of a single real variable $f: \mathbb{R} \rightarrow \mathbb{R}$

1. Select some initial value $x=x_0$ (either randomly or fixed)
2. Select two values of $x$, one to the left, one to the right with a fixed step-size $\Delta$, i.e.
   $x-\Delta$, $x+\Delta$. Compute $f(x), \ f(x-\Delta), \ f(x+\Delta)$.
3. If $f(x)$ is the lowest among them, terminate the algorithm and report $x, \ f(x)$ as min and argmin.
   Otherwise, go either to $x-\Delta$ or $x+\Delta$, depending on where $f$ was smaller.
4. Continue with step $2$ either until the algorithm has stopped (we call that _convergence_) or the maximal amount
   of steps is reached.

Here is what an implementation of the algorithm looks like in python

In [3]:
def greedy_1dim_search(
        f,
        num_steps=50,
        step_size=0.1,
        initial_x=0
):
    current_pos = initial_x
    current_value = f(initial_x)
    for n in range(num_steps):
        step_left, step_right = current_pos - step_size, current_pos + step_size
        value_left, value_right = f(step_left), f(step_right)
        if min(value_left, value_right) >= current_value:
            print(f"Converged after {n} steps")
            return current_pos, current_value

        if value_left < value_right:
            current_pos = step_left
            current_value = value_left
        else:
            current_pos = step_right
            current_value = value_right
    print(f"Failed to converge after {num_steps} steps, returning current candidates for argmin and min")
    return current_pos, current_value

Let us try it out on a bunch of functions

In [15]:
def f1(x):
    return (x-2)**2 + 10

def f2(x):
    return (x - 1.05)**4 - x ** 3

def f3(x):
    return (x-10) ** 2

In [16]:
for f in [f1, f2, f3]:
    print(f"Result for function {f.__name__}")
    print("Min found at {} with value {}".format(*greedy_1dim_search(f)))
    print("")

Result for function f1
Converged after 20 steps
Min found at 2.0000000000000004 with value 10.0

Result for function f2
Converged after 29 steps
Min found at 2.9000000000000012 with value -12.675493750000001

Result for function f3
Failed to converge after 50 steps, returning current candidates for argmin and min
Min found at 4.999999999999998 with value 25.000000000000018



### Note:

There might have been some syntax that you haven't seen before in the above cell. We looped over
a list of functions, used python's inbuilt field `__name__` to extract their names, used _old-style string formatting_
(the new style is using f"..." but it would have been more cumbersome here) and used variable unpacking with `*`.
This is not super important for you right now, but you still should read up on these topics and the syntax to get
more acquainted with python. Just googling and trying things out on your own in a notebook is a good way to go here.

## A note on software engineering

Our implementation of `greedy_1dim_search` works but it is not very well written. We discussed the following questions:
what are the parameters that `greedy_1dim_search` can accept? What does it return?

Python is a _weakly typed_ language, one can entirely omit type information. But it makes the program difficult to read
and also difficult to use correctly if such information is missing. How do we know that `f` should be a function?
What kind of function can it be? How do we know that `greedy_1dim_search` returns a tuple of two floats? This kind of
information is called _signature_ of a function.

In strongly typed languages like Java, C/C++ and others such type information has to be added. It is then checked
when the program is compiled (which happens before it actually runs). In python, one should use the `typing` module
instead to leave hints about the input and return types. These hints are mostly ignored at runtime and thus ado not add
any functionality (there are some exceptions). However, a good programming environment, like PyCharm or VSCode,
will use a type checker to warn you if you try to pass incompatible types. This will make your life much easier and
type hints should therefore always be used! Here is what our function would look like with type hints

In [14]:
def greedy_1dim_search(f: Callable[[float], float],
                       num_steps=50,
                       step_size=0.1,
                       initial_x: float = 0) -> Tuple[float, float]:
    current_pos = initial_x
    current_value = f(initial_x)
    for n in range(num_steps):
        step_left, step_right = current_pos - step_size, current_pos + step_size
        value_left, value_right = f(step_left), f(step_right)
        if min(value_left, value_right) >= current_value:
            print(f"Converged after {n} steps")
            return current_pos, current_value

        if value_left < value_right:
            current_pos = step_left
            current_value = value_left
        else:
            current_pos = step_right
            current_value = value_right
    print(f"Failed to converge after {num_steps} steps, returning current candidates for argmin and min")
    return current_pos, current_value


As you see, nothing has changed in the implementation, only the declaration of the function looks different. Make sure
you google a bit and understand the `Callable` type and all the other ones as well.

## Drawbacks of the greedy search

We discussed problems with the above algorithm and also how to extend it to higher dimensions. The main problems we
identified were:

1. We compute `f` multiple times on the same value. This can easily be fixed in 1 dim. but is harder to fix in more.
2. Because of the fixed step size, we will not find the actual minimum if it does not lie an integer number of step
   sizes away from our initial $x$. This happened for `f2` above.
3. When we are far from the optimum, we need more steps. If we increase the step size, we will have fewer steps but
also lose precision (see point 2 above). This happened for `f3` above.

### Homework 1:
For those missing last time can you see more problems with it? Try to adjust the algorithm such that it works in more
dimensions, i.e. when f is of the type `f: Callable[[float, ...], float]` - a function with an unknown number of
arguments. The initial value for `x` will then be of the type `x: Sequence[float]`. You might need the unpacking
operator `*` for testing your implementation

## Optimization of continuous functions

Many drawbacks arose because we ignored the continuous nature of `f` and just used a discrete search with a fixed
step size. What one can do instead is to use derivatives for finding the right direction to go to and an adjustable
step size simultaneously.

We have then discussed what exactly derivatives are and how to use them for optimization. The resulting algorithm
is called _gradient descent_.

### Homework 2
Especially for those missing last time: please read up on gradient descent. We will discuss it in detail next time
but the discussion will be easier if you are prepared. Also, read as much as possible about derivatives/gradients
and about linear approximations of continuous functions.

Bonus: try to implement gradient descent on your own for some functions of your choice where you know the derivatives
(polynomials would be a good start).

## Preparation for session 3

We will go into more detail about gradient descent, derivatives, optimization and linear models next time.
Python will be our companion for the entirety of the journey, and it is important to master the tool one is using.
I highly recommend you to play around, read and implement as much as you can in preparation for the next class.

For example, [tutorialspoint](https://www.tutorialspoint.com/python/index.htm) and
[w3schools](https://www.w3schools.com/python/) are good places to read about
basic definitions and concepts. There are many more tutorials out there if you prefer something else.

[Hackerrank](https://www.hackerrank.com/) and similar pages are awesome resources for exercises of various difficulties.
I suggest that you create and account there and start solving some problems in python. You can also find tons
of beginners exercises for familiarizing yourselves with syntax, features, data structures and standard libraries.
Get your googling on ;).
